<a href="https://www.bigdatauniversity.com"><img src = "https://ibm.box.com/shared/static/cw2c7r3o20w9zn8gkecaeyjhgw3xdgbj.png" width = 400, align = "center"></a>


<h1 align=center><font size = 5>Decision Trees in R</font></h1>


Hello, and welcome to the Decision Trees in R notebook. In this notebook, we will be going over what Decision Trees are, what they are used for, and how to utilize them in the R environment.

---

## The Classification Problem
Suppose we are lost in a forest, and are very hungry. Unable to go on without eating something first, we take a look around, only to find nothing immediately edible -- just mushrooms.

<center>
<a title="By OliBac from FRANCE [CC BY 2.0 (http://creativecommons.org/licenses/by/2.0)], via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File%3AChampignons_mushrooms_(950475736).jpg"><img width="400" alt="Champignons mushrooms (950475736)" src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/66/Champignons_mushrooms_%28950475736%29.jpg/512px-Champignons_mushrooms_%28950475736%29.jpg"/></a>
<font size="2">*Many different kinds of mushrooms. Are they all safe to eat?*</font>
</center>

We're starving, so anything looks great to us, but eating one of those carelessly might result in us getting poisoned. To know if we can or cannot eat a mushroom, we need to **classify it based on our knowledge of its features**, in other words, we have a classification problem on our hands.

This sort of problem is not simple to solve effectively -- there are many variables involved in correctly classifying something. There are many different kinds of mathematical models that were created to aid us in classification tasks. One of these models is the **Decision Tree model**.

The Decision Tree is a predictive model based on the analysis of a set of data points that describe the type of object we want to classify. In our example, it might be a set of observations of a mushroom's cap type, its color, odor, shape of its stalk, etc. These descriptions of our object are called **features**, and are very important in many different kinds of machine learning algorithms, including Decision Trees. The classification we want out of these features is set aside as a "result" of sorts.

<center>
<img width="680" src="https://ibm.box.com/shared/static/ar8rlcoyrs0n0kphj4g4n4rbhe76vpd9.png"/>
<font size="2">*Example of mushroom features and their classification.*</font>
</center>

From this set of observations, the Decision Tree-creating algorithm infers probabilities -- for example, if many mushrooms that have large caps are poisonous, the algorithm will assume that the probability of large-cap mushrooms being poisonous is high. When the model is complete, we have a tree-like structure composed of what are called **decision nodes**, which ask our data point questions about its features, and **leaf nodes**, which tells us what classification the decision tree thinks our data point is.

<center>
<img width="840" src="https://ibm.box.com/shared/static/urnm2onpitt8qz2296mltzcfdn1p040f.png"/></a>
<font size="2">*Example of a possible Decision Tree describing mushrooms.*</font>
</center>

So, if we create a good Decision Tree, we can follow this "workflow" of sorts to find out what's more probable. Of course, there is no way to be completely certain, but Decision Trees -- when properly created -- are surprisingly effective at what they do.

### The process of constructing the tree
Now, the question is that __how this probablities are calculated?__

let’s quickly review some terminology that can apply to decision trees. If some of these definitions don’t make sense right now, don’t worry, as we’ll be going through some examples that will clarify this.

__Node:__. In a node, we have a dataset that gets tested for a certain attribute. The goal of the node is to split the dataset on an attribute.

__Leaf node__: It is the terminal node in the tree that predicts the outcome.

__Root node__: It appears at the top of tree and contains the entire dataset for that tree.


__ How to find the best feature in each node to split 

__Entropy__: Entropy is calculated for each node. It is the amount of information disorder, or the amount of randomness in the data. The entropy in the node depends on how much random data is in that node. In decision tree we are looking for a trees that have smallest entropy in their nodes. The entropy is used to calculate the homogeneity of the samples in that node. If the samples are completely homogeneous the entropy is zero and if the sample is an equally divided it has entropy of one. It means, if all data in a node are either poisonous or edible, then the entropy is zero, but if the half of data are poisonous and other half are edible, then the entropuy is one. For example, to calculate the Entropy of our target class:  

Entropy = - p(edible)*log(p(edible)) - p(poisonous)*log(p(poisonous))

__ Information gain__: This is the information collected that can increase the level of certainty in a particular predictor (attributes). We can think of information gain and entropy as opposites. As entropy, or the amount of randomness, decreases, the information gain, or amount of certainty, increases, and vice-versa. So, constructing a decision tree is all about finding prdictors (attributes) that returns the highest information gain.

__Algorithm:__

1. Calculate entropy of the target field (the class label) for whole dataset.
2. For each attribute:
    - split the dataset on the attribute
    - calculate entropy of the target field on splited dataset, using the attribute values
    - calculate the information gain of the attribute
3. select the attribute that has the largest informmation gain
4. Branch the tree using the selected attribute
5. stop, if it is a node with entropy of 0, otherwise jump to step2.

we have our dataset at the root node consisting of 4 colors: red, blue, green, and yellow. There are 4 dots for each color, totaling 16 dots.  We’ll be using histograms to look at the chance that an out-of-sample data point would be a certain color (** Click **).



## Decision Trees in R
Now that we know what Decision Trees are, we can move on to creating them in R. To start, let's load up a data set. For this notebook, we will be using <a href=https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/>UCI's `Mushroom` dataset</a> to find out if a mushroom is poisonous or edible. Since this dataset is not inbuilt into R, we need to download it and load it into R. To download the dataset, run the code cell below.

In [ ]:
download.file("https://ibm.box.com/shared/static/dpdh09s70abyiwxguehqvcq3dn0m7wve.data", "mushroom.data")

After you have finished downloading the file, we need to create a data frame to house the observations in the dataset. Since the dataset is structured using comma-separated values, we can use the `read.csv` function.

In [ ]:
mushrooms <- read.csv("mushroom.data", header = F)
mushrooms

Once that's done, we have the data loaded up. However, the way that it is structured isn't the most intuitive. In the code cell below, we are adding the column names to the data frame with the `colnames` function. Additionally, since our data frame is composed of *factors*, we can rename some of these factors to something more easily understood by us using `levels`.

In [ ]:
# Define column names for the mushrooms data frame.
colnames(mushrooms) <- c("Class","cap.shape","cap.surface","cap.color","bruises","odor","gill.attachment","gill.spacing",
                         "gill.size","gill.color","stalk.shape","stalk.root","stalk.surface.above.ring",
                         "stalk.surface.below.ring","stalk.color.above.ring","stalk.color.below.ring","veil.type","veil.color",
                         "ring.number","ring.type","print","population","habitat")
head(mushrooms)

In [ ]:
# Define the factor names for "Class"
levels(mushrooms$Class) <- c("Edible","Poisonous")

In [ ]:
# Define the factor names for "odor"
levels(mushrooms$odor) <- c("Almonds","Anise","Creosote","Fishy","Foul","Musty","None","Pungent","Spicy")
# Define the factor names for "print"
levels(mushrooms$print) <- c("Black","Brown","Buff","Chocolate","Green","Orange","Purple","White","Yellow")
head(mushrooms)

Now we can get to building our model proper. For Decision Trees, we are going to utilize two different, but related, libraries: `rpart` to create the decision tree, and `rpart.plot` to visualize our decision tree. To import libraries, we use the `library` function, like so:

In [ ]:
# Import our required libraries
library(rpart)
library(rpart.plot)

To create our decision tree model, we can use the `rpart` function. `rpart` is simple to use: you provide it a `formula`, show it the dataset it is supposed to use and choose a `method` (either "class" for classification or "anova" for regression).

A great trick to know when handling very large structured datasets (our dataset has over 20 columns we want to use!) is that in `formula` declarations, one can use the `.` operator as a quick way of designating "all other columns" to R. You can also `print` the Decision Tree model to retrieve a summary describing it.

In [ ]:
# Create a classification decision tree using "Class" as the variable we want to predict and everything else as its predictors.
myDecisionTree <- rpart(Class ~ ., data = mushrooms, method = "class")
# Print out a summary of our created model.
print(myDecisionTree)

Now that we have our model, we can draw it to gain a better understanding of how it is classifying the data points. We can use the `rpart.plot` function -- a specialized function for plotting trees -- to render our model. This function takes on some parameters for visualizing the tree in different ways -- try changing the `type` (from 1 to 4) parameter to see what happens!

In [ ]:
rpart.plot(myDecisionTree, type = 3, extra = 2, under = TRUE, faclen=5, cex = .75)

As we can see (under the classification results), our decision tree has perfect accuracy when classifying poisonous mushrooms, and almost perfect accuracy when dealing with edible ones!



In [ ]:
newCase  <- mushrooms[10,-1]
newCase

In [ ]:
predict(myDecisionTree, newCase, type = "class")

### How accurate our model is?

In [ ]:
train_ind <- sample(c(1:n), size = 10)

In [ ]:
## 75% of the sample size
n <- nrow(mushrooms)
smp_size <- floor(0.75 * n)

## set the seed to make your partition reproductible
set.seed(123)
train_ind <- base::sample(c(1:n), size = smp_size)

mushrooms_train <- mushrooms[train_ind, ]
mushrooms_test <- mushrooms[-train_ind, ]

In [ ]:
newDT <- rpart(Class ~ ., data = mushrooms_train, method = "class")

In [ ]:
result <- predict(newDT, mushrooms_test[,-1], type = "class")

In [ ]:
head(result)

In [ ]:
head(mushrooms_test$Class)

In [ ]:
base::table(mushrooms_test$Class, result)

<a id="p"></a>
# <span style="color:red">Practice</span>

<hr>
### <center> Decision Trees - Customer Churn </center>

When do customers give up their loyalty with company X and move to company Y? This is a problem known as **customer churn**, where customers go from one company to a competitor. 

### Customer Churn and Data Science
As a business leader we need to minimize churn. To do so, we need to look at the data to **interpret** why customers churn and build strategies based on theses insights. This is where your job comes in as a **data scientist**. 

In the following scenario, your goal is to explore a dataset to help a telecommunications company reduce customer churn.

## Scenario: Customer Churn in a Telecommunications Company

Below is a sample dataset on customer demographics, their behaviour, and whether or not they end up churning (leaving). This data set provides info to help you predict behavior to retain customers. You can analyze all relevant customer data and develop focused customer retention programs.
A telecommunications company is concerned about the number of customers leaving their landline business for cable competitors. They need to understand who is leaving. Imagine that you’re an analyst at this company and you have to find out who is leaving and why.


### The data set on customer churn includes information about:
- **Customers who left within the last month** – the column is called **Churn**  
- **Services that each customer has signed up for**
  - phone, multiple lines, internet, online security, online backup, - device protection, tech support, and streaming TV and movies
- **Customer account information** 
  - how long they’ve been a customer (tenure), contract, payment method, paperless billing, monthly charges, and total charges  
- **Demographic info about customers**
  - gender, age range, and if they have partners and dependents  


### Getting the Data
The following code cell downloads the data directly onto Data Scientist Workbench using R.

In [ ]:
download.file("https://community.watsonanalytics.com/wp-content/uploads/2015/03/WA_Fn-UseC_-Telco-Customer-Churn.csv", 
             destfile = "/resources/data/custchurn.csv")
print("File saved under /resources/data/custchurn.csv")

In [ ]:
## Your code here



This is the end of the "Decision Trees in R" notebook. Hopefully, now you understand what Decision Trees are, what they are used for, and how to use them in an R environment. Thank you for reading this notebook, and good luck on your studies.

## Want to learn more?

IBM SPSS Modeler is a comprehensive analytics platform that has many machine learning algorithms. It has been designed to bring predictive intelligence to decisions made by individuals, by groups, by systems – by your enterprise as a whole. A free trial is available through this course, available here: [SPSS Modeler for Mac users](https://cocl.us/ML0151EN_SPSSMod_mac) and [SPSS Modeler for Windows users](https://cocl.us/ML0151EN_SPSSMod_win)

Also, you can use Data Science Experience to run these notebooks faster with bigger datasets. Data Science Experience is IBM's leading cloud solution for data scientists, built by data scientists. With Jupyter notebooks, RStudio, Apache Spark and popular libraries pre-packaged in the cloud, DSX enables data scientists to collaborate on their projects without having to install anything. Join the fast-growing community of DSX users today with a free account at [Data Science Experience](https://cocl.us/ML0151EN_DSX)

### Thanks for completing this lesson!

Notebook created by: <a href="https://br.linkedin.com/in/walter-gomes-de-amorim-junior-624726121">Walter Gomes de Amorim Junior</a> , <a href = "https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a>

<hr>
Copyright &copy; 2017 [IBM Cognitive Class](https://cognitiveclass.ai/?utm_source=ML0151&utm_medium=lab&utm_campaign=cclab). This notebook and its source code are released under the terms of the [MIT License](https://cognitiveclass.ai/mit-license/).